In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as A
import albumentations.pytorch
from tqdm.notebook import tqdm

import cv2
import re
import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print ('Available devices ', torch.cuda.device_count())

# print ('Current cuda device ', torch.cuda.current_device())
# print(torch.cuda.get_device_name(device))

# GPU 할당 변경하기
GPU_NUM = 4 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print(device)
print ('Current cuda device ', torch.cuda.current_device()) # check
device_ids = [4,0,2,3]

cuda:4
Current cuda device  4


In [3]:
%time
PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
pretrained_retinanet = model.resnet50(num_classes=80, device=device)
pretrained_retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))


retinanet = model.resnet50(num_classes=5, device=device)
for param, state in zip(pretrained_retinanet.parameters(), pretrained_retinanet.state_dict()) :
    #print(state)
    if 'classificationModel' not in state :
        retinanet.state_dict()[state] = param
    else :
        print(state)
    
for param, state in zip(pretrained_retinanet.fpn.parameters(), pretrained_retinanet.fpn.state_dict()) :
    #print(state)
    retinanet.fpn.state_dict()[state] = param

for param, state in zip(pretrained_retinanet.regressionModel.parameters(), pretrained_retinanet.regressionModel.state_dict()) :
    #print(state)
    retinanet.regressionModel.state_dict()[state] = param  

CPU times: user 5 µs, sys: 8 µs, total: 13 µs
Wall time: 27.9 µs


In [4]:

# retinanet.to(device)
retinanet = torch.nn.DataParallel(retinanet, device_ids = [4,0,2,3], output_device=4).to(device)
# retinanet = DataParallelModel(retinanet, device_ids = device_ids)
retinanet.to(device)
# retinanet.cuda()
retinanet.module.freeze_bn()

In [5]:
retinanet.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [6]:
train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 32
train_ds = PapsDataset(train_info, transforms)

train_data_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

In [7]:
criterion = FocalLoss(device)
# criterion = DataParallelCriterion(criterion, device_ids = device_ids) 
criterion = criterion.to(device)
# optimizer = optim.Adam(retinanet.parameters(), lr=1e-4)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
# retinanet.train()
retinanet.training = True
# retinanet.module.freeze_bn()    
# retinanet.freeze_bn()

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(retinanet.parameters(), lr = 1e-7)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=1, eta_max=0.0001,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [8]:
# #for i, data in enumerate(tqdm(train_data_loader)) :
# EPOCH_NUM = 60
# loss_per_epoch = 0.5
# for epoch in range(EPOCH_NUM) :
#     epoch_loss = []
#     total_loss = 0
#     tk0 = tqdm(train_data_loader, total=len(train_data_loader), leave=False)
#     EPOCH_LEARING_RATE = optimizer.param_groups[0]["lr"]
#     print("*****{}th epoch, learning rate {}".format(epoch, EPOCH_LEARING_RATE))
    
#     for step, data in enumerate(tk0) :
#         images, _, paths, targets = data
# #         print(targets)
#         batch_size = len(images)

#     #     images = list(image.to(device) for image in images)
#         c, h, w = images[0].shape
#         images = torch.cat(images).view(-1, c, h, w).to(device)
# #         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#         targets = [ t.to(device) for t in targets]

# #         classification_loss, regression_loss = retinanet([images, targets])
#         outputs = retinanet([images, targets])
#         classification, regression, anchors, annotations = (outputs)
#         classification_loss, regression_loss = criterion(classification, regression, anchors, annotations)

# #         output = retinanet(images)
# #         features, regression, classification = output
# #         classification_loss, regression_loss = criterion(classification, regression, modified_anchors, targets)    
#         classification_loss = classification_loss.mean()
#         regression_loss = regression_loss.mean()
#         loss = classification_loss + regression_loss 
#         total_loss += loss.item()
        
#         epoch_loss.append((loss.item()))
#         tk0.set_postfix(lr=optimizer.param_groups[0]["lr"], batch_loss=loss.item(), cls_loss=classification_loss.item(), 
#                         reg_loss=regression_loss.item(), avg_loss=total_loss/(step+1))

#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)
#         optimizer.step()   

#     print('{}th epochs loss is {}'.format(epoch, np.mean(epoch_loss)))
#     if loss_per_epoch > np.mean(epoch_loss):
#         print('best model is saved')
#         torch.save(retinanet.state_dict(), 'best_model.pt')
#         loss_per_epoch = np.mean(epoch_loss)
# #     scheduler.step(np.mean(epoch_loss))
#     scheduler.step()

# torch.save(retinanet.state_dict(), '../trained_models/model.pt')


In [9]:
test_info = np.load('../data/test.npy', allow_pickle=True, encoding='latin1').item()
test_ds = PapsDataset(test_info,val_transforms)

test_data_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [10]:
#dataset_val = COCO('../data/coco/val.json')
dataset_val = CocoDataset('../data/', set_name='val',
                                  transform=val_transforms, isPapsmear=True)
print(len(dataset_val))

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
3739


In [11]:
#dataset_val[0]

In [12]:
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import json
import torch

retinanet.eval()
# retinanet.training = True
tk1 = tqdm(test_data_loader, total=len(test_data_loader),leave=False)

# start collecting results
results = []
image_ids = []
threshold = 0.05
with torch.no_grad():
#for step, data in enumerate(tk1) :
    for index in range(len(dataset_val)):
        data = dataset_val[index]
        scale = data['scale']
        print(data)

        images, _, path, targets = data
#         print(path)
        batch_size = len(images)
        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)
#         print(images.shape)
        targets = [ t.to(device) for t in targets]   
        
        scores, labels, boxes = retinanet(images)
        if boxes.shape[0] > 0:
            # change to (x, y, w, h) (MS COCO standard)
            boxes[:, 2] -= boxes[:, 0]
            boxes[:, 3] -= boxes[:, 1]     
            
            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(boxes.shape[0]):
                score = float(scores[box_id])
                label = int(labels[box_id])
                box = boxes[box_id, :]

                # scores are sorted, so we can break
                if score < threshold:
                    break

                # append detection for each positively labeled class
                image_result = {
#                     'image_id'    : dataset.image_ids[index],
#                     'category_id' : dataset.label_to_coco_label(label),
                    'image_id'    : path[0],
                    'category_id' : label,
                    'score'       : float(score),
                    'bbox'        : box.tolist(),
                }

                # append detection to results
                results.append(image_result)    
                
        # append image to list of processed images
        image_ids.append(path)
        if step > 100 :
            break
        
if not len(results):
    print("no results at all")

# write output
json.dump(results, open('../data/papsmear_bbox_results.json', 'w'), indent=4)
    

  0%|          | 0/3744 [00:00<?, ?it/s]

(1800, 4000, 3)
anno [[1978.  867. 2068.  996.    5.]]


ValueError: Your 'label_fields' are not valid - them must have same names as params in dict

In [ ]:
# results

In [ ]:
# # df = pd.read_csv('df.csv')
# test_info = np.load('../data/test.npy', allow_pickle=True, encoding='latin1').item()

In [ ]:
# paps_GT[0]

In [ ]:
# len(test_info['/home/Dataset/Papsmear/original/SS2/03(200908-10-normal)/20200908_093508(0).jpg'])

In [ ]:
test_info
paps_GT = []
for path in test_info.keys():
    label = test_info[path]
    for i in range(len(label)) :
        paps_gt = {
                'image_id'    : path,
                'category_id' : label[i][5],
                'score'       : 1,
                'bbox'        : box.tolist(),
            }
        paps_GT.append(paps_gt) 
        
json.dump(paps_GT, open('../data/papsmear_GT.json', 'w'), indent=4)   


In [ ]:
coco = COCO('../data/coco/val.json')
print(type(coco))

In [ ]:
# coco.dataset

In [ ]:
# paps_pred = coco.loadRes('../data/papsmear_bbox_results.json')
paps_gt = coco.loadRes('../data/papsmear_bbox_results.json')

In [ ]:
# with open('../data/papsmear_bbox_results.json', 'r') as f:
#     paps_pred = json.load(f)
# with open('../data/papsmear_GT.json', 'r') as f:
#     paps_gt = json.load(f)    


In [ ]:
# run COCO evaluation
coco_eval = COCOeval(paps_gt, paps_pred, 'bbox')
coco_eval.params.imgIds = image_ids
# coco_eval.params.catIds = [1]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

In [ ]:
paps_gt